In [5]:
#Defining the variables:
#1. Places == rooms
#2. Passages == dors
#3. Characters
#4. ¿Objects? == keys

old_man = {
    "name": "old man",
    "type": "character",
}

dock = {
    "name": "dock",
    "type": "passage",
}

contract = {
    "name": "contract to embark ship El Rei",
    "type": "key",
    "target": dock,
}

cask = {
    "name": "wine cask",
    "type": "furniture",
}


lisboa = {
    "name": "Lisboa",
    "type": "room",
}

beach = {
    "name": "beach",
    "type": "passage",
}

supplies = {
    "name": "fresh supplies",
    "type": "key",
    "target": beach,
}

chest = {
    "name": "rusty chest",
    "type": "furniture",
}

supplier = {
    "name": "tall odd fellow",
    "type": "character",
}

cabo_verde = {
    "name": "Cabo Verde",
    "type": "room",
}

neptune = {
    "name": "King Neptune",
    "type": "passage",
}

tribute = {
    "name": "tribute to the king",
    "type": "key",
    "target": neptune,
}

storm = {
    "name": "storm",
    "type": "room",
}

destination = {
    "name": "Brasil"
}

all_rooms = [lisboa, cabo_verde, storm, destination]

all_doors = [dock, beach, neptune]

# define which items/rooms are related

object_relations = {
    #rooms
    "Lisboa": [old_man, cask, dock],
    "Cabo Verde": [beach, supplier, chest],
    "storm": [neptune],
    "Brasil": [beach],
    #key keepers
    "old man": [contract],
    "tall odd fellow": [supplies],
    "rusty chest": [tribute],
    #passages
    "dock": [lisboa, cabo_verde],
    "beach": [cabo_verde, storm],
    "King Neptune": [storm, destination]
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": lisboa,
    "keys_collected": [],
    "keys_removed": [],
    "target_room": destination
}

In [10]:
#libraries

from PIL import Image

import winsound

ModuleNotFoundError: No module named 'winsound'

In [11]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
                                                                                                                #CHANGES
    myImage = Image.open("/Users/ritaoliveira/Documents/curso_Ironhack/EXERCISES/The-game/departure.jpg")
    myImage.show()
    winsound.PlaySound('pirates_of_the_caribbean_theme.mp3',winsound.SND_ASYNC)
    print("You wake up on top of what it looks like a pile of bags. You remember something about the night before, you were drinking with friends, they all called you crazy, but hugged you as they said their goodbyes. Why would they say goodbye to you? You look around and see small boats departing towards a big fleet… THUNDERS! You’re supposed to be in that fleet! You are supposed to go to India but now your late. Hurry up and get yourself on one of those boats!")
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! It's not India but you made it to Brazil!")
    else:
        print("You are now in " + room["name"])
        
        intended_action = input("What would you like to do? Type 'explore' or 'examine'?").strip()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine?").strip())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the place. This is " + room["name"] + ". You find " + ", ".join(items)) 

def get_next_room_of_door(passage, current_room):
    """
    From object_relations, find the two rooms connected to the given passage.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[passage["name"]]                                
    for room in connected_rooms:
        if(not current_room == room):         
            return room

def examine_item(item_name):
    """
    Examine an item which can be a passage, a character or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a passage. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    
    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "passage"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        targ_key = key["name"]
                        have_key = True
                if(have_key):
                    
                    output += "You unlock it with a "+ targ_key + "."                        #Changed here
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have yet the key."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")
    
    if(next_room and input("Do you want to go to the next room? Enter 'yes' or 'no'").strip() == 'yes'):

        statement_of_room(next_room)                                                      #CHANGES
        play_room(next_room)
    else:
        play_room(current_room)
                                                                                           #CHANGES
def statement_of_room(next_room):
    if next_room["name"] == "Cabo Verde":
        print("After thirteen days of bad sea, rain and cold wind on the ship El Rei, you are able to step, for the first time in your life, a land that is not Portugal. You are in São Nicolau, Cape Verde and you’ve been given the task to find the a supplier that will provide you fresh provisions for the remaining trip. Don’t take to much time or you will be punished with cleaning the deck for a month again! ")
        myImage1 = Image.open("../The-game/Planta_CV.jpg")
        myImage1.show()
    elif next_room["name"] == "storm":
        print("A big storm hits from nowhere, the crew runs to their designed roles, the timonner struggles to keep a safe route. Suddenly a rope whips a blow in the air, the main sail just falls while it was retracted, the crew hope and morale just cracks as they watch powerless their doom. Everyone knows the main mast will not resist the wind... You hear someone screaming braking through the roaming storm, and more screams while a giant wave form at the starboard of the ship. You try your best to ignore, imaging your mates being throwed by the gusty winds just to been swallowed by the raging waves... When a mate pulls your shoulder and screams: ‘Neptune! King Neptune is here!’")
        print("\n")
        myImage2 = Image.open("../The-game/storm.jpg")
        myImage2.show()
        print("King Neptune: ‘Who is fullish enough to enter my waters without permission? I demand a sacrifice! If acceptable then I shall grant you steady winds and a safe sail, if not I will feed your bodies to my children and imprison your souls to my trident for eternity!.")
    else:
        pass

In [12]:
game_state = INIT_GAME_STATE.copy()

start_game()

NameError: name 'winsound' is not defined